In [ ]:
import numpy as np
import pandas as pd
import lightgbm as lgb
import datetime, itertools, zipfile, os, time, psutil, gc
from datetime import date, timedelta
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import mean_squared_error, roc_auc_score
from sklearn.preprocessing import LabelEncoder
import sys
sys.path.append('../input/my-temp-files/')
import alml

RS = alml.rs(0)

SUBMIT = False
SUBMIT = True

ROUNDS = 1000

def rmse(y_true, y_pred):
	score = np.sqrt(mean_squared_error(y_true, y_pred))
	return score

NROWS = None
NROWS_VAL = None
if not SUBMIT:
	NROWS = 30000 #None 
	NROWS_VAL = 20000

MODELS = ['lgb']
#MODELS = ['lgb','rfr','lnr','lasso','ridge','abr','etr','gbr']

MODELS2 = ['lgb']
LEVEL = 1 

BAGS = 0 if SUBMIT else 0
FOLDS = 5 if SUBMIT else 5
EARLY_STOPPING = 100 #100

if LEVEL == 2:
	FOLDS = 0
	MODELS = MODELS2

# XGB
params1 = {}
params1['objective'] = 'binary:logistic'		#'binary:logistic'
params1['eval_metric'] = 'auc'			#'logloss'
params1['eta'] = 0.04
params1['max_depth'] = 7
params1['silent'] = 1
params1['subsample'] = 0.8
params1['subsample_bytree'] = 0.7
#params1['subsample_bylevel'] = 0.7
params1['num_rounds'] = 30 #100

#LightGBM
params2 = {}
params2['objective'] = 'regression' #'regression'
params2['metric'] = 'rmse'		#'multi_logloss', 'binary_logloss'
params2['boosting'] = 'gbdt' #'gbdt'
params2['learning_rate'] = 0.05 #0.2
params2['verbose'] = -1
#params2['num_class'] = 6
params2['num_leaves'] = 15 # 1:2 2:4 3:8 4:16 5:32 6:64 7:128 8:256
params2['bagging_fraction'] = 0.8
params2['bagging_freq'] = 1
params2['bagging_seed'] = RS
params2['feature_fraction'] = 0.7
#params2['colsample_bytree'] = 0.05
params2['feature_fraction_seed'] = RS
#params2['scale_pos_weight'] = 10
#params2['max_bin'] = 300
params2['max_depth'] = 5 #6 #s-1
params2['min_child_samples'] = 90
#params2['min_gain_to_split'] = 0.5
params2['reg_lambda'] = 2
params2['reg_alpha'] = 0.4
params2['num_rounds'] = ROUNDS
#params2['tree_learner'] = 'voting'
params2['feature_name'] = 'auto'
#params2['categorical_feature'] = ["region", "city", "parent_category_name", "category_name", "user_type", "param_1", "param_2", "param_3"]
#params2['histogram_pool_size'] = 8000

params3 = {}
params3['n_jobs'] = 2
params3['n_estimators'] = 100
params3['max_depth'] = 5
params3['verbose'] = 0
params3['learning_rate'] = 0.1

if LEVEL == 2:
	#XGB
	ROUNDS = 980

CONFIG = []
if 'lgb' in MODELS:
	CONFIG.append({'model': 'lgb','params': params2,})
if 'etr' in MODELS:
	CONFIG.append({'model': 'etr','params': params3,})
if 'abr' in MODELS:
	CONFIG.append({'model': 'abr','params': params3,})
if 'gbr' in MODELS:
	CONFIG.append({'model': 'gbr','params': params3,})
if 'xgb' in MODELS:
	CONFIG.append({'model': 'xgb','params': params1,})
if 'rfr' in MODELS:
	CONFIG.append({'model': 'rfr','params': params3,})
if 'lasso' in MODELS:
	CONFIG.append({'model': 'lasso','params': params3,})
if 'ridge' in MODELS:
	CONFIG.append({'model': 'ridge','params': params3,})
if 'lnr' in MODELS:
	CONFIG.append({'model': 'lnr','params': params3,})

print("Started {}".format("SUBMIT" if SUBMIT else "TEST"))
print(psutil.virtual_memory())
target = 'target'
scorefn = rmse
feats_not_to_use = ['card_id', target, 'first_active_month', 'fold_id','merchant_id']
cat_vars = [] 

tr = pd.read_csv('../input/elo-merchant-category-recommendation/train.csv') [['card_id','target']]
print('Train  : shape=',tr.shape)

trm = tr['target'].mean()
trtmp = tr[['target']].copy()
trtmp['preds'] = trm
scoremean = scorefn(trtmp['target'], trtmp['preds'])
print('Score mean:',scoremean)
del trtmp, trm

#, usecols=['card_id','merchant_id','purchase_date','purchase_amount','installments','month_lag']
df = pd.read_csv('../input/elo-merchant-category-recommendation/historical_transactions.csv', nrows=NROWS, parse_dates=['purchase_date']).sample(frac=0.8)
print('History: shape=',df.shape)
print(df.dtypes)

df = df.sort_values(['card_id','purchase_date'])

df['next_card_id'] = df['card_id'].shift(-1)
df['prev_card_id'] = df['card_id'].shift(1)
df['next_date'] = df['purchase_date'].shift(-1)
df['prev_date'] = df['purchase_date'].shift(1)
df['next_amount'] = df['purchase_amount'].shift(-1)
df['prev_amount'] = df['purchase_amount'].shift(1)
print('a')
df['next_merchant_id'] = df['merchant_id'].shift(-1)
df['prev_merchant_id'] = df['merchant_id'].shift(1)
df['next_installments'] = df['installments'].shift(-1)
df['prev_installments'] = df['installments'].shift(1)
df['next_category_2'] = df['category_2'].shift(-1)
df['prev_category_2'] = df['category_2'].shift(1)
print('b')
df['next_category_3'] = df['category_3'].shift(-1)
df['prev_category_3'] = df['category_3'].shift(1)
df.loc[df['next_card_id'] != df['card_id'], 'next_date'] = np.nan
df.loc[df['card_id'] != df['prev_card_id'], 'prev_date'] = np.nan
df.loc[df['next_card_id'] != df['card_id'], 'next_amount'] = np.nan
df.loc[df['card_id'] != df['prev_card_id'], 'prev_amount'] = np.nan
df.loc[df['next_card_id'] != df['card_id'], 'next_merchant_id'] = np.nan
df.loc[df['card_id'] != df['prev_card_id'], 'prev_merchant_id'] = np.nan
df.loc[df['next_card_id'] != df['card_id'], 'next_installments'] = np.nan
df.loc[df['card_id'] != df['prev_card_id'], 'prev_installments'] = np.nan
df.loc[df['next_card_id'] != df['card_id'], 'next_category_2'] = np.nan
df.loc[df['card_id'] != df['prev_card_id'], 'prev_category_2'] = np.nan
df.loc[df['next_card_id'] != df['card_id'], 'next_category_3'] = np.nan
df.loc[df['card_id'] != df['prev_card_id'], 'prev_category_3'] = np.nan

df['next_merchant_same'] = (df['next_merchant_id'] == df['merchant_id']).astype(int)
#df['prev_merchant_same'] = (df['prev_merchant_id'] == df['merchant_id']).astype(int)
df.drop(['next_merchant_id','prev_merchant_id','city_id','merchant_category_id','state_id','subsector_id'], axis=1, inplace=True)
print('c')

df['days_to_next'] = (df['next_date'] - df['purchase_date']).dt.days
df['days_from_prev'] = (df['purchase_date'] - df['prev_date']).dt.days
df['hours_to_next'] = np.round((df['next_date'] - df['purchase_date']).dt.seconds * 60 * 60)
df['hours_from_prev'] = np.round((df['purchase_date'] - df['prev_date']).dt.seconds * 60 * 60)
print('d')

df['amount_vs_next'] = df['purchase_amount'] / df['next_amount']
#df['amount_vs_prev'] = df['purchase_amount'] / df['prev_amount']
df['installments_vs_next'] = df['installments'] / (df['next_installments'] + 0.001)
df['installments_vs_prev'] = df['installments'] / (df['prev_installments'] + 0.001)
print('e')

df = df[['card_id','merchant_id','purchase_date','purchase_amount','installments','month_lag','days_to_next','days_from_prev','hours_to_next'\
		,'hours_from_prev','next_merchant_same','authorized_flag','category_1','category_2','category_3'\
		,'next_installments','prev_installments','next_category_2','prev_category_2','next_category_3','prev_category_3'\
		,'amount_vs_next','installments_vs_next','installments_vs_prev']]
#,'amount_vs_prev','prev_merchant_same'
print(df.head(100).tail(50))

df = alml.proc_reduce_mem(df)

#df['authorized_flag'] = df['authorized_flag'].map({'Y':1, 'N':0})
df['category_2'] = df['category_2'].astype(str)

df['date_uptonow'] = (pd.Timestamp('2018-12-06') - df['purchase_date']).dt.days
df['month_uptonow'] = ((pd.Timestamp('2018-12-06') - df['purchase_date']).dt.days)//30
df['date_diff'] = df['date_uptonow'] + df['month_lag']*30	
#df['month_diff'] = df['month_uptonow'] + df['month_lag']	


df['year'] = df['purchase_date'].dt.year
df['weekofyear'] = df['purchase_date'].dt.weekofyear
df['month'] = df['purchase_date'].dt.month
#df['dayofweek'] = df['purchase_date'].dt.dayofweek
#df['weekend'] = (df.purchase_date.dt.weekday >=5).astype(int)
#df['hour'] = df['purchase_date'].dt.hour
df['purchase_date'] = df['purchase_date'].astype(np.int64) * 1e-9

df = alml.proc_reduce_mem(df)

df = df.merge(tr, how='left', on='card_id')
print('Merged : shape=',df.shape)

def procfeats(df):

	df['installments_vs_month_lag'] = df['installments'] / (df['month_lag'] + 0.001)
	return df
	
df = procfeats(df)

print(df.dtypes)
cat_vars = [f for f in df.columns if df[f].dtype == 'object' and f not in feats_not_to_use]
print('Categorical:',cat_vars)

for col in cat_vars:
	lbl = LabelEncoder()
	df[col] = lbl.fit_transform(df[col].values.astype('str'))

print(df.head())
	
x_train = df[~df['target'].isnull()].reset_index(drop=True)
x_test = df[df['target'].isnull()].reset_index(drop=True)
del df
gc.collect()

use = [x for x in x_train.columns.values if x not in feats_not_to_use]
cid = x_train['card_id'].values
train_y = x_train[target].values
#x_train = x_train[use]
cid_test = x_test['card_id'].values
x_test = x_test[use]

print('Training on',x_train.shape, train_y.shape ,'testing on',x_test.shape)	
params2['categorical_feature'] = [x for x in cat_vars if x in use]
print('Categorical:',params2['categorical_feature'])	

gc.collect()
print(psutil.virtual_memory())
if 0:
	alml.features_eliminator(x_train, train_y, CONFIG[0]['params'], seed=RS, bags=2, score_fn=scorefn, logname='features_eliminator.txt')
	print('elimination done')
	exit()
	
if 1:	
	foldsdf = pd.read_csv('../input/my-temp-files/train_folds_1.csv')
	print(foldsdf.head())
	print(foldsdf.shape)
	print(x_train.shape)
	x_train = x_train.merge(foldsdf, how='left', on='card_id')
	print(x_train.shape)
	preds_test = None
	preds_train = None
	for ff in range(FOLDS):
		params2['num_rounds'] = ROUNDS

		print('FOLD:', ff)
		fold_train = x_train[x_train['fold_id'] != ff].copy()
		fold_cv = x_train[x_train['fold_id'] == ff].copy()
		fold_train_y = fold_train[target].values
		fold_cv_y = fold_cv[target].values
		fold_cv_ids = fold_cv['card_id'].values
		
		fold_train = fold_train[use]
		fold_cv = fold_cv[use]
		
		preds, preds_cv, models, preds_cv_cols = alml.ml(fold_train, fold_train_y, x_test, CONFIG, None, True, BAGS, 0, score_fn=scorefn, stratify=True, early_stopping=EARLY_STOPPING, Xval=fold_cv, yval=fold_cv_y)
		preds_test = preds if preds_test is None else preds_test + preds
		
		cvd = pd.DataFrame({'card_id':fold_cv_ids, 'preds':preds_cv, 'target':fold_cv_y})
		print(cvd.head())
		print(cvd.shape)
		print('Score:',scorefn(cvd['target'], cvd['preds']))
		preds_train = cvd if preds_train is None else pd.concat([preds_train, cvd])
	preds = preds_test / FOLDS
else:
	preds, preds_cv, models, preds_cv_cols = alml.ml(x_train, train_y, x_test, CONFIG, None, True, BAGS, FOLDS, score_fn=scorefn, stratify=False, early_stopping=EARLY_STOPPING)
	
del x_train
gc.collect()
	
df_train = preds_train
print('Score full on',df_train.shape[0],'entries',rmse(df_train[target],df_train['preds']))
dfg = df_train.groupby('card_id').mean().reset_index()
print('Score grouped mean on',dfg.shape[0],'entries',rmse(dfg[target],dfg['preds']))
dfg = df_train.groupby('card_id').min().reset_index()
print('Score grouped min on',dfg.shape[0],'entries',rmse(dfg[target],dfg['preds']))
dfg = df_train.groupby('card_id').max().reset_index()
print('Score grouped max on',dfg.shape[0],'entries',rmse(dfg[target],dfg['preds']))
print('Score mean baseline:',scoremean)

print(df_train.describe())

print('\n----- biggest errors')
df_train['error'] = (df_train['preds'] - df_train[target]) ** 2
print(df_train.sort_values('error', ascending=False).head(30))

print('\n----- lowest preds')
print(df_train.sort_values('preds', ascending=True).head(30))

now = str(datetime.datetime.now().strftime("%m%d%H%M"))
tf = 's1_preds_%s.' % now
	
if not SUBMIT:
	savename = 'cv_sub_last.csv'
	print('\nPreparing to save CV',df_train.shape)
	print(df_train.head())
	df_train.to_csv(savename, index=False)
	print('CV saved to',savename)

if SUBMIT:
	savename = 'cv_' + tf + 'csv'
	print('\nPreparing to save CV',df_train.shape)
	print(df_train.head())
	df_train.to_csv(savename, index=False)
	print('CV saved to',savename)

	print('\nTest predictions preparing...')
	savename = 'test_' + tf + 'csv'
	cvd = pd.DataFrame({'card_id':cid_test, 'preds':preds})
	print(cvd.head())
	print(cvd.shape)
	print(cvd.describe())
	cvd.to_csv(savename, index=False)
	print('Test saved to',savename)
	
print("\nDone.")